In [36]:
import pandas as pd
import folium
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [37]:
import folium
import requests
import json
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [38]:
df=pd.read_csv('C:/Users/ebuka/mergeddataset.csv')

In [39]:
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [40]:
len(df.Borough.unique())

10

In [41]:
df.Borough.value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East York            5
York                 5
East Toronto         5
Mississauga          1
Name: Borough, dtype: int64

In [43]:
address = 'ontario, Toronto'
#this is used to get lat and long of an adress

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

GeocoderTimedOut: Service timed out

In [44]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [45]:
#CLIENT_ID = 'SHZGKF5EWPSH2Q1I0T0J5PCHBQCGPK0CVZRQK1ZAFH2ZBRAF' # your Foursquare ID
#CLIENT_SECRET = 'KXPHK4RZ3AENYNT3GOEO11G5Y4EXM1RWNBI202VL0YC443ZT' # your Foursquare Secret
#VERSION = '20180605' # Foursquare API version
#LIMIT=100
#radius=500

#url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#    CLIENT_ID, 
 #   CLIENT_SECRET, 
  #  VERSION, 
   # neighborhood_latitude, 
    #neighborhood_longitude, 
    #radius, 
    #LIMIT)

In [46]:
dft=pd.get_dummies(df)

In [47]:
km=KMeans(n_clusters=5)
ar=km.fit(dft)

In [48]:
ar.labels_

array([0, 0, 1, 0, 1, 3, 4, 0, 1, 1, 0, 3, 4, 0, 1, 1, 2, 3, 4, 1, 1, 2,
       4, 2, 1, 1, 4, 0, 0, 1, 1, 2, 4, 0, 0, 1, 1, 2, 4, 0, 0, 1, 1, 2,
       4, 0, 0, 1, 1, 0, 0, 4, 0, 0, 1, 0, 2, 0, 4, 0, 0, 2, 2, 2, 2, 4,
       0, 2, 2, 2, 3, 4, 0, 2, 2, 2, 2, 3, 4, 2, 1, 2, 4, 2, 1, 4, 2, 1,
       3, 3, 4, 1, 1, 3, 3, 4, 1, 1, 3, 1, 1, 3, 3])

In [49]:
df['labels']=ar.labels_

In [50]:
df

,Postal Code,Borough,Neighborhood,Latitude,Longitude,labels
0,M3A,North York,Parkwoods,43.753259,-79.329656,0
1,M4A,North York,Victoria Village,43.725882,-79.315572,0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1
...,...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,3
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,1
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,1
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,3


In [51]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(5)
ys = [i + x + (i*x)**2 for i in range(5)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df['Latitude'], df['Longitude'], df['Neighborhood'], df['labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters